# NotebookLM adding sources automatically

## Firstly login to Google account and save the profile in the specific directory which will be used for next runs

In [ ]:
from playwright.async_api import async_playwright
import asyncio
import os

profile_path = os.path.expanduser("~/.browser_use_redhat_profile")

async def main():
    async with async_playwright() as p:
        # This is where your cookies and login sessions will be stored
        browser = await p.chromium.launch_persistent_context(
            user_data_dir=profile_path,
            headless=False,
        )
        page = await browser.new_page()
        await page.goto("https://accounts.google.com")

        print("Please log in manually and then close the browser window when done.")
        await page.wait_for_timeout(60000 * 10)  # 10 minutes to log in

await main()


## Add links as sources (Website, Youtube)

In [ ]:
from playwright.async_api import async_playwright
import asyncio
import os

profile_path = os.path.expanduser("~/.browser_use_redhat_profile")

notebook_url = "https://notebooklm.google.com/notebook/5e4bd5cb-9b98-4ff4-b0dc-4abccb02e861"
links = [
    "https://access.redhat.com/support/policy/updates/openshift",
    "https://access.redhat.com/support/policy/rhel-container-compatibility",
    "https://access.redhat.com/support/policy/updates/containertools",
    "https://access.redhat.com/support/policy/updates/hybridcloud-console/lifecycle",
    "https://www.youtube.com/watch?v=b9BWbr_7xs8",
    "https://www.youtube.com/watch?v=IX9DW49XiQo",
]


async def main():
    async with async_playwright() as p:
        # This is where your cookies and login sessions will be stored
        browser = await p.chromium.launch_persistent_context(
            user_data_dir=profile_path,
            executable_path="/usr/bin/chromium-browser",
            headless=False, # Set to True if you want to run in the background
        )
        page = await browser.new_page()
        await page.goto(notebook_url)


        for link in links:
            # Wait for the page to load and the "Add source" button to be visible
            # Using text content as a locator can be robust to some UI changes
            await page.locator("text='Add'").wait_for(state='visible')

            # Click the "Add source" button
            await page.locator("text='Add'").click()

            # Wait for the source options to appear and click "Webpage" or "Youtube"
            # Again, using text content
            if "youtube.com" in link:
                text_to_click = "YouTube"
            else:
                text_to_click = "Website"
            await page.locator(f"text='{text_to_click}'").wait_for(state='visible')
            await page.locator(f"text='{text_to_click}'").click()

            # --- NEW APPROACH: Locate input field relative to the "Paste URL" label within the modal ---
            # Selector for the modal container based on the provided HTML
            modal_selector = '.mat-mdc-dialog-inner-container'

            # Wait for the modal container to be visible
            await page.locator(modal_selector).wait_for(state='visible', timeout=15000)

            # Now, locate the input field within this modal by finding the label
            # and navigating to the associated input within its form field container.
            # This chain finds the modal, then the mat-label with specific text,
            # goes up to its ancestor mat-form-field, and finds the input inside.
            if "youtube.com" in link:
                text_to_fill = "Paste YouTube URL"
            else:
                text_to_fill = "Paste URL"
            url_input_locator = page.locator(modal_selector).locator(f"mat-label:text('{text_to_fill}')").locator('xpath=ancestor::mat-form-field').locator('input')

            # Fill the input field. Playwright's fill() waits for the element to be actionable.
            print("Attempting to fill URL input...")
            # Use a robust timeout for the fill action itself
            await url_input_locator.fill(link, timeout=20000)
            print("Filled URL input.")

            # Locate the "Insert" button *within* the modal
            # We find the button that contains the text "Insert"
            insert_button_selector = f"{modal_selector} button:has-text('Insert')"

            # Click the "Insert" button. Playwright's click() waits for the element to be actionable (including enabled).
            print("Attempting to click Insert button...")
            await page.locator(insert_button_selector).click(timeout=20000)
            print("Clicked Insert button.")
            # --- END OF NEW APPROACH ---

            print(f"Attempted to add webpage: {link}")

            # You might want to add a wait here to see the result or for the source to be processed
            await page.wait_for_timeout(2000) # Wait for 5 seconds

        await browser.close()

await main()